# Hackathon

Some utilities

## Import Utils

In [3]:
import keras
from keras.metrics import top_k_categorical_accuracy
import h5py as h5
import numpy as np
import pandas as pd
import tables

PATH_DATA = '/notebooks/data/full.h5'
OUT_PATH = '/notebooks/data/balanced_full.h5'

In [19]:
classes = [1, 2, 3, 4, 5, 10, 11, 12, 14, 19, 20, 17]

nb_per_class = np.zeros(20)

In [4]:
image_batch = np.zeros((100000, 16, 16, 4))

In [23]:
##DataSet Full
top_landcover = h5.File(PATH_DATA)['TOP_LANDCOVER']
top_landcover_batch = top_landcover[300000:400000].astype(int)
image_batch = h5.File(PATH_DATA)['S2'][300000:400000, :, :, :]

In [24]:
##Traitement des labels : 23 -> 14
unique, counts = np.unique(top_landcover_batch, return_counts=True)
print(unique)
print(counts)

[ 1  2  3  4  5 10 11 12 14 20]
[19804 36816 26923   117   437     5    34 15652    13   199]


In [20]:
nb_per_class[19] = 100000
nb_per_class[0] += 34732
nb_per_class[1] += 39382
nb_per_class[2] += 18914
nb_per_class[3] += 173
nb_per_class[4] += 774
nb_per_class[9] += 8
nb_per_class[10] += 20
nb_per_class[11] += 5273
nb_per_class[13] += 10
nb_per_class[18] += 4
nb_per_class[19] += 704



[3.47320e+04 3.93820e+04 1.89140e+04 1.73000e+02 7.74000e+02 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 8.00000e+00 2.00000e+01 5.27300e+03
 0.00000e+00 1.00000e+01 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 4.00000e+00 1.00704e+05]


In [21]:
print (nb_per_class.astype(int))

[ 34732  39382  18914    173    774      0      0      0      0      8
     20   5273      0     10      0      0      0      0      4 100704]


In [28]:
classes = [1, 2, 3, 4, 5, 10, 11, 12, 14, 19, 20, 17]
indexes = np.zeros((len(classes), 200000))

for i in range(len(classes)):
        current_class = classes[i]       
        if current_class != 17:
            indexes[i][:] = np.where(top_landcover == current_class)[0][:200000]
        else:
             indexes[i][:86471] = np.where(top_landcover == current_class)[0][:]
                
indexs = np.sort(indexes.flatten()[:2286471])  

1
2
3
4
5
10
11
12
14
19
20
17


In [ ]:
## DataSet Equilibré
balanced_top_lancover = h5.File(PATH_DATA)['TOP_LANDCOVER'][indexs,:]
balanced_images = h5.File(PATH_DATA)['S2'][indexs,:,:,:]

In [ ]:
import os 

def build_h5_file(top_lancover, images, h5_output_path):
    if os.path.exists(h5_output_path):
        os.remove(h5_output_path)
    f = h5.File(h5_output_path, 'w')
    top_landcover_submit = f.create_dataset("TOP_LANDCOVER", (len(top_lancover), 1), maxshape=(None, 1))
    top_landcover_submit[:, 0] = top_lancover[:,0]
    s2_submit = f.create_dataset("S2", (len(images), 16,16,4), maxshape=(None, 16,16,4))
    s2_submit[:,:,:,:] = images
    f.close()
    
    return 1

build_h5_file(top_landcover_batch, balanced_images, OUT_PATH)

In [22]:
import pickle

f_batch = open("/notebooks/data/landcover_batch2", 'wb')
pickle.dump(top_landcover_batch, f_batch)
f_batch.close()

f_batch_images = open("/notebooks/data/images_batch2", 'wb')
pickle.dump(image_batch, f_batch_images)
f_batch_images.close()